# DEBUG FUNCTIONS

In [2]:

from collections import Counter

from typing import Dict, List, Tuple
import json
import pandas as pd

# Percorsi dei file
DEV_PATH = "../../data/subtask_a_dev.json"
PRED_PATH = "../predictions/subtask_a_dev_bert_token_classification_preds_extended.json"

print("Paths set:")
print("  DEV_PATH :", DEV_PATH)
print("  PRED_PATH:", PRED_PATH)

Paths set:
  DEV_PATH : ../../data/subtask_a_dev.json
  PRED_PATH: ../predictions/subtask_a_dev_bert_token_classification_preds_extended.json


In [3]:
def load_jsonl(path: str):
    """Load a JSON lines file or JSON array file."""
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read().strip()
    if not text:
        return []
    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        data = []
        for line in text.splitlines():
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data


def build_sentence_gold_map(records):
    """Convert dataset rows into list of sentences with aggregated terms."""
    out = {}
    
    if isinstance(records, dict) and 'data' in records:
        rows = records['data']
    else:
        rows = records
    
    for r in rows:
        key = (r.get('document_id'), r.get('paragraph_id'), r.get('sentence_id'))
        if key not in out:
            out[key] = {
                'document_id': r.get('document_id'),
                'paragraph_id': r.get('paragraph_id'),
                'sentence_id': r.get('sentence_id'),
                'sentence_text': r.get('sentence_text', ''),
                'terms': []
            }
        
        if isinstance(r.get('term_list'), list):
            for t in r.get('term_list'):
                if t and t not in out[key]['terms']:
                    out[key]['terms'].append(t)
        else:
            term = r.get('term')
            if term and term not in out[key]['terms']:
                out[key]['terms'].append(term)
    
    return list(out.values())


def preprocess_text(text, force_lower=True):
    # fix encoding issues
    text = text.replace("\u00a0", " ")

    # normalize spaces
    text = " ".join(text.split())

    # unify apostrophes
    text = text.replace("’", "'").replace("`", "'")

    # lowercase if model is uncased
    if force_lower:
        text = text.lower()

    # remove weird control characters
    text = "".join(c for c in text if c.isprintable())

    return text


### Carica dev gold e fai lo stesso cleaning del training

In [4]:
# Carica dev gold
dev_data = load_jsonl(DEV_PATH)
dev_sentences = build_sentence_gold_map(dev_data)

# Applica lo stesso preprocess di training
for entry in dev_sentences:
    entry["sentence_text"] = preprocess_text(entry["sentence_text"])
    entry["terms"] = [preprocess_text(t) for t in entry["terms"]]

print(f"Dev sentences: {len(dev_sentences)}")
print("Esempio:")
print("  Text :", dev_sentences[0]['sentence_text'])
print("  Terms:", dev_sentences[0]['terms'])

Dev sentences: 577
Esempio:
  Text : non domestica; campeggi, distributori carburanti, parcheggi; 1,22; 4,73
  Terms: []


In [5]:
def load_predictions(path: str):
    with open(path, "r", encoding="utf-8") as f:
        preds_raw = json.load(f)
    return preds_raw


def build_pred_map(preds_raw) -> Dict[Tuple[str, int, int], List[str]]:
    """
    Restituisce una mappa:
        (document_id, paragraph_id, sentence_id) -> lista di termini predetti (stringhe)
    Tenta di essere robusta a formati del tipo:
      - {"data": [{..., "term_list": [...]}, ...]}
      - [{"document_id": ..., "term_list": [...]}, ...]
      - campi 'term_list', 'predicted_terms', 'bert_terms', ecc.
    """
    if isinstance(preds_raw, dict) and "data" in preds_raw:
        rows = preds_raw["data"]
    else:
        rows = preds_raw

    candidate_term_keys = ["term_list", "predicted_terms", "bert_terms", "pred_terms"]

    pred_map: Dict[Tuple[str, int, int], List[str]] = {}

    for r in rows:
        doc_id = r.get("document_id")
        par_id = r.get("paragraph_id")
        sent_id = r.get("sentence_id")

        if doc_id is None or par_id is None or sent_id is None:
            raise ValueError(f"Manca uno tra document_id / paragraph_id / sentence_id in record: {r}")

        # Prova a trovare il campo che contiene la lista di termini
        term_list = None
        for key in candidate_term_keys:
            if key in r:
                term_list = r[key]
                break

        if term_list is None:
            raise ValueError(
                f"Nessun campo termini trovato in record {r}. "
                f"Attesi uno tra: {candidate_term_keys}"
            )

        if term_list is None:
            term_list = []

        # Normalizza a lista di stringhe
        term_list = [str(t) for t in term_list]

        pred_map[(doc_id, par_id, sent_id)] = term_list

    return pred_map


preds_raw = load_predictions(PRED_PATH)
pred_map = build_pred_map(preds_raw)

print(f"Loaded predictions for {len(pred_map)} sentences")


Loaded predictions for 577 sentences


In [6]:
def normalize_term_for_eval(t: str) -> str:
    # usa lo stesso cleaning che usi altrove
    return preprocess_text(t)


def align_gold_pred(dev_sentences, pred_map):
    """
    Ritorna due liste parallele:

      gold_lists: [ [gold_term1, gold_term2, ...],  ... ]
      pred_lists: [ [pred_term1, pred_term2, ...],  ... ]

    dove ogni elemento corrisponde a UNA frase di dev_sentences, e i termini
    sono già normalizzati.
    """
    gold_lists = []
    pred_lists = []

    missing_keys = []

    for entry in dev_sentences:
        key = (entry["document_id"], entry["paragraph_id"], entry["sentence_id"])

        gold_terms = [normalize_term_for_eval(t) for t in entry["terms"]]

        if key not in pred_map:
            # Nessuna predizione per questa frase
            missing_keys.append(key)
            pred_terms = []
        else:
            pred_terms_raw = pred_map[key]
            pred_terms = [normalize_term_for_eval(t) for t in pred_terms_raw]

        gold_lists.append(gold_terms)
        pred_lists.append(pred_terms)

    if missing_keys:
        print(f"WARNING: {len(missing_keys)} frasi nel dev non hanno predizioni nel file.")
        print("Esempio chiave mancante:", missing_keys[0])

    return gold_lists, pred_lists


In [7]:
gold_lists, pred_lists = align_gold_pred(dev_sentences, pred_map)

print("Aligned gold and predictions:")
print(f"  gold_lists: {len(gold_lists)}")
print(f"  pred_lists: {len(pred_lists)}")


Aligned gold and predictions:
  gold_lists: 577
  pred_lists: 577


In [19]:
def build_sentence_error_table(dev_sentences, gold_lists, pred_lists) -> pd.DataFrame:
    rows = []
    for idx, (entry, gold_terms, pred_terms) in enumerate(zip(dev_sentences, gold_lists, pred_lists)):
        gold_set = set(gold_terms)
        pred_set = set(pred_terms)

        missing = gold_set - pred_set
        extra   = pred_set - gold_set
        # stampo SOLO dove ci sono errori reali
        if missing or extra:
            rows.append({
                "idx": idx,
                "document_id": entry["document_id"],
                "paragraph_id": entry["paragraph_id"],
                "sentence_id": entry["sentence_id"],
                "n_gold": len(gold_set),
                "n_pred": len(pred_set),
                "n_missing": len(missing),
                "n_extra": len(extra),
                "missing_terms": list(missing),
                "extra_terms": list(extra),
            })

    df = pd.DataFrame(rows)
    return df


errors_df = build_sentence_error_table(dev_sentences, gold_lists, pred_lists)
print(f"Error table shape: {errors_df.shape}")
errors_df.tail(50)

Error table shape: (134, 10)


,idx,document_id,paragraph_id,sentence_id,n_gold,n_pred,n_missing,n_extra,missing_terms,extra_terms
84,356,doc_salerno_05,24,7,4,3,1,0,"[plastica, acciaio e alluminio]",[]
85,364,doc_poggiomarino_12,17,50,0,2,0,2,[],"[erba, rifiuto]"
86,370,doc_santegidiodelmontealbino_03,15,2,6,5,3,2,"[banda stagnata, vanno conferiti, latta]","[metalli, conferiti]"
87,371,doc_praiano_02,8,1,2,3,2,3,"[lattine in alluminio con il simbolo al, metal...","[lattine, metallo, alluminio]"
88,374,doc_santagnello_16,38,0,1,0,1,0,[dichiarazione tari],[]
89,381,doc_caserta_02,51,3,1,0,1,0,[tarsu],[]
90,382,doc_poggiomarino_01,8,11,1,0,1,0,[contenitori siglati t/f/x],[]
91,384,doc_gragnano_03,5,0,3,3,1,1,[piano di riorganizzazione del servizio di rac...,[servizio di raccolta rsu]
92,387,doc_capaccio_10,9,4,4,3,2,1,"[carta - cartone - tetra pak, busta con legaccio]",[carta]
93,388,doc_salerno_06,12,5,3,1,2,0,"[utenti, interruzione programmata del servizio]",[]


### Debug di una frase specifica (testo + gold + pred + missing/extra)

In [9]:
def debug_sentence_by_index(idx: int, dev_sentences, gold_lists, pred_lists):
    entry = dev_sentences[idx]
    gold_terms = list(set(gold_lists[idx]))
    pred_terms = list(set(pred_lists[idx]))

    gold_set = set(gold_terms)
    pred_set = set(pred_terms)

    missing = gold_set - pred_set
    extra   = pred_set - gold_set

    print("=" * 100)
    print(f"Sentence index: {idx}")
    print(f"Document: {entry['document_id']} | Paragraph: {entry['paragraph_id']} | Sentence: {entry['sentence_id']}")
    print("\nTEXT:")
    print(entry["sentence_text"])

    print("\nGOLD TERMS:")
    print(gold_terms)

    print("\nPREDICTED TERMS:")
    print(pred_terms)

    print("\nMISSING TERMS (gold not predicted):")
    print(list(missing))

    print("\nEXTRA TERMS (predicted but not in gold):")
    print(list(extra))


# Esempio: debuggare la prima frase con qualche errore
rows_with_errors = errors_df[(errors_df["n_missing"] > 0) | (errors_df["n_extra"] > 0)]
if not rows_with_errors.empty:
    example_idx = int(rows_with_errors.iloc[0]["idx"])
    debug_sentence_by_index(example_idx, dev_sentences, gold_lists, pred_lists)
else:
    print("Non ci sono frasi con missing/extra terms (improbabile ma bellissimo!)")

Sentence index: 1
Document: doc_caserta_06 | Paragraph: 3 | Sentence: 1

TEXT:
il presente disciplinare per la gestione dei centri di raccolta comunali è stato redatto ai sensi e per effetto del dm 13/05/2009, pubblicato sulla g.u. n. 165 del 18/07/2009, con il quale sono state apportate le modifiche sostanziali al dm 08/04/2008, disciplina dei centri di raccolta dei rifiuti urbani raccolti in modo differenziato, come previsto dall'art. 183, comma 7, lettera cc) del dlgs 3 aprile 2006, n. 152, e ss.mm.ii.

GOLD TERMS:
['disciplinare per la gestione dei centri di raccolta comunali', 'disciplina dei centri di raccolta dei rifiuti urbani raccolti in modo differenziato']

PREDICTED TERMS:
['disciplina', 'centri di raccolta dei rifiuti urbani raccolti', 'gestione', 'centri di raccolta comunali', 'disciplinare']

MISSING TERMS (gold not predicted):
['disciplinare per la gestione dei centri di raccolta comunali', 'disciplina dei centri di raccolta dei rifiuti urbani raccolti in modo differenz

### Debug di frasi “peggiori” (con più missing / extra)

In [ ]:
def debug_top_k_errors(errors_df, dev_sentences, gold_lists, pred_lists, k: int = 5):
    df_sorted = errors_df.sort_values(["n_missing", "n_extra"], ascending=False)
    top = df_sorted.head(k)
    for _, row in top.iterrows():
        idx = int(row["idx"])
        debug_sentence_by_index(idx, dev_sentences, gold_lists, pred_lists)


# Esempio: ispeziona le 5 frasi peggiori
debug_top_k_errors(errors_df, dev_sentences, gold_lists, pred_lists, k=10)

Sentence index: 186
Document: doc_poggiomarino_12 | Paragraph: 17 | Sentence: 65

TEXT:
r1 frigoriferi e sistemi per il condizionamento – r2 lavatrici, lavastoviglie, cucine, scaldabagni – r3 tv e monitor – r4 piccoli elettrodomestici, hardware da information tecnology, elettronica di consumo, apparecchi illuminanti – r5 sorgenti luminose.

GOLD TERMS:
['r1', 'r4', 'frigoriferi e sistemi per il condizionamento', 'r3', 'lavatrici, lavastoviglie, cucine, scaldabagni', 'r5', 'r2']

PREDICTED TERMS:
['elettronica di']

MISSING TERMS (gold not predicted):
['r1', 'frigoriferi e sistemi per il condizionamento', 'r4', 'r3', 'lavatrici, lavastoviglie, cucine, scaldabagni', 'r5', 'r2']

EXTRA TERMS (predicted but not in gold):
['elettronica di']
Sentence index: 272
Document: doc_sorrento_22 | Paragraph: 2 | Sentence: 0

TEXT:
oltre 400 chilogrammi di rifiuti raccolti, corrispondenti a circa venti buste di multimateriale, una transenna arrugginita, un fusto di ferro con altri materiali ferrosi, u